# GenGSR-NET

## Things to change before running the Jupyter Notebook
1. Ensure that the following Python packages are installed:
    - `numpy`
    - `random`
    - `torch`
    - `pandas`
    - `sklearn`
    - `seaborn`
    - `matplotlib`
    - `scipy`
    - `networkx`
2. In the third cell change the filepath to the directory where the data is stored.

In [ ]:
import random
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon
import networkx as nx
import community as community_louvain
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure correct installation of community-louvain library
!pip uninstall -y community
!pip uninstall -y python-louvain
!pip install python-louvain

## Set the seed for reproducibility

In [ ]:

# Set a fixed random seed for reproducibility across multiple libraries
random_seed = 24
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

## Helpers

In [ ]:
import numpy as np


class MatrixVectorizer:
    """
    A class for transforming between matrices and vector representations.

    This class provides methods to convert a symmetric matrix into a vector (vectorize)
    and to reconstruct the matrix from its vector form (anti_vectorize), focusing on
    vertical (column-based) traversal and handling of elements.
    """

    def __init__(self):
        """
        Initializes the MatrixVectorizer instance.

        The constructor currently does not perform any actions but is included for
        potential future extensions where initialization parameters might be required.
        """
        pass

    @staticmethod
    def vectorize(matrix, include_diagonal=False):
        """
        Converts a matrix into a vector by vertically extracting elements.

        This method traverses the matrix column by column, collecting elements from the
        upper triangle, and optionally includes the diagonal elements immediately below
        the main diagonal based on the include_diagonal flag.

        Parameters:
        - matrix (numpy.ndarray): The matrix to be vectorized.
        - include_diagonal (bool, optional): Flag to include diagonal elements in the vectorization.
          Defaults to False.

        Returns:
        - numpy.ndarray: The vectorized form of the matrix.
        """
        # Determine the size of the matrix based on its first dimension
        matrix_size = matrix.shape[0]

        # Initialize an empty list to accumulate vector elements
        vector_elements = []

        # Iterate over columns and then rows to collect the relevant elements
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Collect upper triangle elements
                        vector_elements.append(matrix[row, col])
                    elif include_diagonal and row == col + 1:
                        # Optionally include the diagonal elements immediately below the diagonal
                        vector_elements.append(matrix[row, col])

        return np.array(vector_elements)

    @staticmethod
    def anti_vectorize(vector, matrix_size, include_diagonal=False):
        """
        Reconstructs a matrix from its vector form, filling it vertically.

        The method fills the matrix by reflecting vector elements into the upper triangle
        and optionally including the diagonal elements based on the include_diagonal flag.

        Parameters:
        - vector (numpy.ndarray): The vector to be transformed into a matrix.
        - matrix_size (int): The size of the square matrix to be reconstructed.
        - include_diagonal (bool, optional): Flag to include diagonal elements in the reconstruction.
          Defaults to False.

        Returns:
        - numpy.ndarray: The reconstructed square matrix.
        """
        # Initialize a square matrix of zeros with the specified size
        matrix = np.zeros((matrix_size, matrix_size))

        # Index to keep track of the current position in the vector
        vector_idx = 0

        # Fill the matrix by iterating over columns and then rows
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Reflect vector elements into the upper triangle and its mirror in the lower triangle
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1
                    elif include_diagonal and row == col + 1:
                        # Optionally fill the diagonal elements after completing each column
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1

        return matrix

## Load Data
Change the file path to the path of the data on your machine.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/DGL_kaggle/Cluster-CV .

In [ ]:

# Change the file path to the location of the data

data_hr_split_1 = pd.read_csv('Cluster-CV/Fold1/hr_clusterA_modified.csv')
data_hr_split_1_np = data_hr_split_1.to_numpy()

data_lr_split_1 = pd.read_csv('Cluster-CV/Fold1/lr_clusterA.csv')
data_lr_split_1_np = data_lr_split_1.to_numpy()

data_hr_split_2 = pd.read_csv('Cluster-CV/Fold2/hr_clusterB_modified.csv')
data_hr_split_2_np = data_hr_split_2.to_numpy()

data_lr_split_2 = pd.read_csv('Cluster-CV/Fold2/lr_clusterB.csv')
data_lr_split_2_np = data_lr_split_2.to_numpy()

data_hr_split_3 = pd.read_csv('Cluster-CV/Fold3/hr_clusterC_modified.csv')
data_hr_split_3_np = data_hr_split_3.to_numpy()

data_lr_split_3 = pd.read_csv('Cluster-CV/Fold3/lr_clusterC.csv')
data_lr_split_3_np = data_lr_split_3.to_numpy()

test_data = pd.read_csv('Cluster-CV/Fold1/hr_clusterA_modified.csv')
test_data_np = test_data.to_numpy()

In [ ]:
def weight_variable_glorot(output_dim):

    input_dim = output_dim
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = np.random.uniform(-init_range, init_range, (input_dim, output_dim))

    return initial

In [ ]:
def pad_HR_adj(label, split):

    label = np.pad(label, ((split, split), (split, split)), mode="constant")
    np.fill_diagonal(label, 1)
    return torch.from_numpy(label).type(torch.FloatTensor)

def normalize_adj_torch(mx):
    rowsum = mx.sum(1)
    r_inv_sqrt = torch.pow(rowsum, -0.5).flatten()
    r_inv_sqrt[torch.isinf(r_inv_sqrt)] = 0.0
    r_mat_inv_sqrt = torch.diag(r_inv_sqrt)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    mx = torch.transpose(mx, 0, 1)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    return mx

def random_zero_rows_columns(adj_matrix, fraction=0.1):
    """
    Randomly zero out a fraction of rows and the corresponding columns in an adjacency matrix.

    Parameters:
    - adj_matrix: PyTorch tensor representing the adjacency matrix.
    - fraction: Fraction of rows/columns to zero out.

    Returns:
    - Modified adjacency matrix with randomly selected rows and columns zeroed out.
    """
    n = adj_matrix.size(0)
    num_to_zero = int(n * fraction)
    indices = np.random.choice(n, num_to_zero, replace=False)

    modified_adj_matrix = adj_matrix.clone()
    modified_adj_matrix[indices] = 0
    modified_adj_matrix[:, indices] = 0

    return modified_adj_matrix

def unpad(data, split):

    idx_0 = data.shape[0] - split
    idx_1 = data.shape[1] - split
    train = data[split:idx_0, split:idx_1]
    return train


In [ ]:
class GSRLayer(nn.Module):
    def __init__(self, hr_dim):
        super(GSRLayer, self).__init__()
        self.hr_dim = hr_dim
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Initialize weights on the correct device
        initial_weights = weight_variable_glorot(hr_dim)
        self.weights = torch.nn.Parameter(torch.FloatTensor(initial_weights).to(self.device), requires_grad=True)

    def forward(self, A, X):
        lr = A.to(self.device)
        f = X.to(self.device)

        eig_val_lr, U_lr = torch.linalg.eigh(lr, UPLO='U')

        eye_mat = torch.eye(lr.shape[0], device=self.device)
        s_d = torch.cat((eye_mat, eye_mat), 0)[:self.hr_dim]

        a = torch.matmul(self.weights, s_d)
        b = torch.matmul(a, U_lr.t())
        f_d = torch.abs(torch.matmul(b, f))
        f_d.fill_diagonal_(1)

        adj = normalize_adj_torch(f_d)
        X = torch.mm(adj, adj.t())
        X = (X + X.t()) / 2
        idx = torch.eye(self.hr_dim, dtype=torch.bool, device=self.device)
        X[idx] = 1

        return adj, torch.abs(X)


class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.5, act=F.relu):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.act = act
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Initialize weights on the correct device
        self.weight = torch.nn.Parameter(torch.FloatTensor(in_features, out_features).to(self.device))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        input = input.to(self.device)
        adj = adj.to(self.device)

        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)

        return output

In [ ]:
class GAT(nn.Module):
    """
    A basic implementation of the GAT layer.

    This layer applies an attention mechanism in the graph convolution process,
    allowing the model to focus on different parts of the neighborhood
    of each node.
    """
    def __init__(self, in_features, out_features, activation):
        super(GAT, self).__init__()
        # Initialize the weights, bias, and attention parameters as
        # trainable parameters
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.zeros(out_features))
        self.phi = nn.Parameter(torch.FloatTensor(2 * out_features, 1))
        self.activation = activation
        self.reset_parameters()


    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

        stdv = 1. / np.sqrt(self.phi.size(1))
        self.phi.data.uniform_(-stdv, stdv)


    def forward(self, input, adj):
        # Apply linear transformation and add bias
        h = torch.mm(input, self.weight) + self.bias


        h_expanded = h.unsqueeze(1)
        h_repeated = h.unsqueeze(0)

        concatenated = torch.cat((h_expanded.repeat(1, adj.size(0), 1),
                                  h_repeated.repeat(adj.size(0), 1, 1)), dim=-1)

        attention_scores = torch.matmul(concatenated, self.phi).squeeze(-1)  # Shape: (N, N)

        mask = adj + torch.eye(adj.size(0), device=adj.device)

        S_masked = torch.where(mask > 0, attention_scores, torch.tensor(float('-inf'), device=attention_scores.device))

        attention_weights = F.softmax(S_masked, dim=1)

        h_prime = torch.mm(attention_weights, h)

        if self.activation is not None:
            h = self.activation(h_prime)
        else:
            h = h_prime

        return h

In [ ]:
class GraphUnpool(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GraphUnpool, self).__init__()
        self.gat = GAT(in_features=in_dim, out_features=out_dim, activation=F.relu)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.gat.to(self.device)

    def forward(self, A, X, idx):
        new_X = torch.zeros([A.shape[0], X.shape[1]], device=self.device)
        new_X[idx] = X
        new_X = self.gat(new_X, A)  # Apply GAT after unpooling to the features
        return A, new_X

class GraphPool(nn.Module):
    def __init__(self, k, in_dim, out_dim):
        super(GraphPool, self).__init__()
        self.k = k
        self.gat = GAT(in_features=in_dim, out_features=out_dim, activation=F.relu)
        self.proj = nn.Linear(out_dim, 1)  # Note: Using out_dim from GAT output
        self.sigmoid = nn.Sigmoid()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.gat.to(self.device)
        self.proj.to(self.device)

    def forward(self, A, X):
        X = X.to(self.device)
        X = self.gat(X, A)  # Process X through GAT before pooling
        scores = self.proj(X)
        scores = torch.squeeze(scores)
        scores = self.sigmoid(scores)
        num_nodes = A.shape[0]
        values, idx = torch.topk(scores, int(self.k * num_nodes))
        new_X = X[idx, :]
        values = torch.unsqueeze(values, -1)
        new_X = torch.mul(new_X, values)
        A = A[idx, :]
        A = A[:, idx]
        return A, new_X, idx


class GCN(nn.Module):
    def __init__(self, in_dim, out_dim, activation=F.relu):
        super(GCN, self).__init__()
        self.gat = GAT(in_features=in_dim, out_features=out_dim, activation=activation)
        self.proj = nn.Linear(out_dim, out_dim)
        self.drop = nn.Dropout(p=0.5)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, A, X):
        X = X.to(self.device)
        X = self.drop(X)
        X = self.gat(X, A)
        X = self.proj(X)
        return X


class GraphUnet(nn.Module):
    def __init__(self, ks, in_dim, out_dim, dim=268):
        super(GraphUnet, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.start_gcn = GCN(in_dim, dim).to(self.device)
        self.bottom_gcn = GCN(dim, dim).to(self.device)
        self.end_gcn = GCN(2 * dim, out_dim).to(self.device)
        self.down_gcns = nn.ModuleList([GCN(dim, dim).to(self.device) for _ in range(len(ks))])
        self.up_gcns = nn.ModuleList([GCN(dim, dim).to(self.device) for _ in range(len(ks))])
        self.pools = nn.ModuleList([GraphPool(k, dim, dim).to(self.device) for k in ks])
        self.unpools = nn.ModuleList([GraphUnpool(dim, dim).to(self.device) for _ in range(len(ks))])
        self.l_n = len(ks)

    def forward(self, A, X):
        A = A.to(self.device)
        X = X.to(self.device)
        adj_ms = []
        indices_list = []
        down_outs = []

        X = self.start_gcn(A, X)
        start_gcn_outs = X
        org_X = X

        for i in range(self.l_n):
            X = self.down_gcns[i](A, X)
            adj_ms.append(A)
            down_outs.append(X)
            A, X, idx = self.pools[i](A, X)
            indices_list.append(idx)

        X = self.bottom_gcn(A, X)

        for i in range(self.l_n):
            up_idx = self.l_n - i - 1
            A, idx = adj_ms[up_idx], indices_list[up_idx]
            A, X = self.unpools[i](A, X, idx)
            X = self.up_gcns[i](A, X)
            X = X.add(down_outs[up_idx])

        X = torch.cat([X, org_X], 1)
        X = self.end_gcn(A, X)

        return X, start_gcn_outs


In [ ]:
class GSRNet(nn.Module):
    def __init__(self, ks, args):
        super(GSRNet, self).__init__()


        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.lr_dim = args.lr_dim
        self.hr_dim = args.hr_dim
        self.hidden_dim = args.hidden_dim


        self.layer = GSRLayer(self.hr_dim).to(self.device)
        self.net = GraphUnet(ks, self.lr_dim, self.hr_dim).to(self.device)
        self.gc1 = GraphConvolution(self.hr_dim, self.hidden_dim, 0, act=F.relu).to(self.device)
        self.gc2 = GraphConvolution(self.hidden_dim, self.hr_dim, 0, act=F.relu).to(self.device)

    def forward(self, lr):
        lr = lr.to(self.device)
        I = torch.eye(self.lr_dim, device=self.device)
        A = normalize_adj_torch(lr)
        A = random_zero_rows_columns(A, fraction=0.4)

        self.net_outs, self.start_gcn_outs = self.net(A, I)

        self.outputs, self.Z = self.layer(A, self.net_outs)

        self.hidden1 = self.gc1(self.Z, self.outputs)
        self.hidden2 = self.gc2(self.hidden1, self.outputs)

        z = self.hidden2
        z = (z + z.transpose(0, 1)) / 2

        idx = torch.eye(self.hr_dim, dtype=torch.bool, device=self.device)
        z[idx] = 1

        return torch.abs(z), self.net_outs, self.start_gcn_outs, self.outputs


In [ ]:
criterion = nn.MSELoss()

def train(model, optimizer, subjects_adj, subjects_labels, args, val_adj, val_ground_truth):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    i = 0
    all_epochs_loss = []
    val_epochs_loss = []
    no_epochs = args.epochs

    # Early stopping parameters
    early_stopping_patience = 10
    early_stopping_min_delta = 0.0001
    min_training_epochs = 100

    # Early stopping variables
    best_loss = float('inf')
    epochs_no_improve = 0
    early_stop = False
    epoch = 0

    while epoch < min_training_epochs or not early_stop:
        epoch_loss = []
        epoch_error = []

        for lr, hr in zip(subjects_adj, subjects_labels):

            model.train()
            optimizer.zero_grad()

            lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
            hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)

            model_outputs, net_outs, start_gcn_outs, layer_outs = model(lr)
            model_outputs = unpad(model_outputs, args.padding)

            padded_hr = pad_HR_adj(hr.cpu(), args.padding).to(device)
            eig_val_hr, U_hr = torch.linalg.eigh(hr, UPLO='U')


            loss = (
                args.lmbda * criterion(net_outs, start_gcn_outs)
                + criterion(model.layer.weights, U_hr)
                + criterion(model_outputs, hr)
            )

            error = criterion(model_outputs, hr)

            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.item())
            epoch_error.append(error.item())

        mean_epoch_loss = np.mean(epoch_loss)
        all_epochs_loss.append(np.mean(epoch_loss))

        model.eval()
        val_loss = []
        with torch.no_grad():
            for lr, hr in zip(val_adj, val_ground_truth):
                lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
                hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)

                model_outputs, net_outs, start_gcn_outs, layer_outs = model(lr)
                model_outputs = unpad(model_outputs, args.padding)

                padded_hr = pad_HR_adj(hr.cpu(), args.padding).to(device)
                eig_val_hr, U_hr = torch.linalg.eigh(hr, UPLO='U')

                loss = (
                    args.lmbda * criterion(net_outs, start_gcn_outs)
                    + criterion(model.layer.weights, U_hr)
                    + criterion(model_outputs, hr)
                )

                val_loss.append(loss.item())

        mean_val_loss = np.mean(val_loss)
        val_epochs_loss.append(mean_val_loss)

        if mean_val_loss < best_loss - early_stopping_min_delta:
            best_loss = mean_val_loss
            epochs_no_improve = 0
            torch.save(model, f'./Result/model_fold_{fold_num}.pth')
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience and epoch >= min_training_epochs:
            early_stop = True

        epoch += 1

        i += 1
        print(f"Epoch: {epoch}/{no_epochs}, Training Loss: {mean_epoch_loss}, Error: {np.mean(epoch_error):.4f}, Val Loss: {mean_val_loss}")

    return all_epochs_loss, val_epochs_loss


def test(model, test_adj, test_labels, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    test_error = []
    preds_list = []
    g_t = []

    i = 0
    # TESTING
    for lr, hr in zip(test_adj, test_labels):

        all_zeros_lr = not np.any(lr)
        all_zeros_hr = not np.any(hr)

        if (
            all_zeros_lr == False and all_zeros_hr == False
        ):  # choose representative subject
            lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
            np.fill_diagonal(hr, 1)
            hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)
            preds, a, b, c = model(lr)
            preds = unpad(preds, args.padding)
            preds_flattened = preds.flatten().cpu().detach().numpy()
            preds_matrix = MatrixVectorizer.anti_vectorize(preds_flattened, args.hr_dim)
            preds_list.append(preds_matrix)

            error = criterion(preds, hr)
            g_t.append(hr.flatten().cpu().numpy())
            #print(error.item())
            test_error.append(error.item())

            i += 1

    print("Test error MSE: ", np.mean(test_error))
    return np.array(preds_list)


In [ ]:
matrix_vec = MatrixVectorizer()

In [ ]:
def convert_dataset(data, matrix_dim):
    num_samples = data.shape[0]
    matrices = np.zeros((num_samples, matrix_dim, matrix_dim))

    for i in range(num_samples):
        vector = data[i]
        matrix = matrix_vec.anti_vectorize(vector, matrix_dim)
        matrices[i] = matrix

    return matrices

In [ ]:
data_hr_split_1_reshaped = convert_dataset(data_hr_split_1_np, 268)
data_lr_split_1_reshaped = convert_dataset(data_lr_split_1_np, 160)

data_hr_split_2_reshaped = convert_dataset(data_hr_split_2_np, 268)
data_lr_split_2_reshaped = convert_dataset(data_lr_split_2_np, 160)

data_hr_split_3_reshaped = convert_dataset(data_hr_split_3_np, 268)
data_lr_split_3_reshaped = convert_dataset(data_lr_split_3_np, 160)

In [ ]:
import numpy as np


def data(data_lr_reshaped, data_hr_reshaped):
    # Determine the number of samples for training based on an 80-20 split
    num_samples = data_lr_reshaped.shape[0]
    train_size = int(num_samples * 0.8)

    # Split the data into training and testing datasets
    subjects_adj = data_lr_reshaped[:train_size]
    subjects_labels = data_hr_reshaped[:train_size]

    test_adj = data_lr_reshaped[train_size:]
    test_labels = data_hr_reshaped[train_size:]

    return subjects_adj, subjects_labels, test_adj, test_labels

def data_es(data_lr_reshaped, data_hr_reshaped):
    # use the last 20 samples for early stopping
    num_samples = data_lr_reshaped.shape[0]
    train_size = int(num_samples - 20)

    subjects_adj = data_lr_reshaped[:train_size]
    subjects_labels = data_hr_reshaped[:train_size]

    test_adj = data_lr_reshaped[train_size:]
    test_labels = data_hr_reshaped[train_size:]

    return subjects_adj, subjects_labels, test_adj, test_labels


subjects_adj_split_1, subjects_labels_split_1, test_adj_split_1, test_labels_split_1 = data_es(
    data_lr_split_1_reshaped, data_hr_split_1_reshaped
)
subjects_adj_split_2, subjects_labels_split_2, test_adj_split_2, test_labels_split_2 = data_es(
    data_lr_split_2_reshaped, data_hr_split_2_reshaped
)
subjects_adj_split_3, subjects_labels_split_3, test_adj_split_3, test_labels_split_3 = data_es(
    data_lr_split_3_reshaped, data_hr_split_3_reshaped
)

In [ ]:
def evaluation_metrics(num_test_samples, pred_matrices, gt_matrices):
    # post-processing
    pred_matrices[pred_matrices < 0] = 0

    gt_matrices[gt_matrices < 0] = 0

    # Initialize lists to store MAEs for each centrality measure
    mae_bc = []
    mae_ec = []
    mae_pc = []

    # Initialize lists to store flattened matrices
    pred_1d_list = []
    gt_1d_list = []

    # Iterate over each test sample
    for i in range(num_test_samples):
        # Convert adjacency matrices to NetworkX graphs
        pred_graph = nx.from_numpy_array(pred_matrices[i], edge_attr="weight")
        gt_graph = nx.from_numpy_array(gt_matrices[i], edge_attr="weight")

        # Compute centrality measures
        pred_bc = nx.betweenness_centrality(pred_graph, weight="weight")
        pred_ec = nx.eigenvector_centrality(pred_graph, weight="weight")
        pred_pc = nx.pagerank(pred_graph, weight="weight")

        gt_bc = nx.betweenness_centrality(gt_graph, weight="weight")
        gt_ec = nx.eigenvector_centrality(gt_graph, weight="weight")
        gt_pc = nx.pagerank(gt_graph, weight="weight")

        # Convert centrality dictionaries to lists
        pred_bc_values = list(pred_bc.values())
        pred_ec_values = list(pred_ec.values())
        pred_pc_values = list(pred_pc.values())

        gt_bc_values = list(gt_bc.values())
        gt_ec_values = list(gt_ec.values())
        gt_pc_values = list(gt_pc.values())

        # Compute MAEs
        mae_bc.append(mean_absolute_error(pred_bc_values, gt_bc_values))
        mae_ec.append(mean_absolute_error(pred_ec_values, gt_ec_values))
        mae_pc.append(mean_absolute_error(pred_pc_values, gt_pc_values))

        # Vectorize matrices
        pred_1d_list.append(MatrixVectorizer.vectorize(pred_matrices[i]))
        gt_1d_list.append(MatrixVectorizer.vectorize(gt_matrices[i]))

    # Compute average MAEs
    avg_mae_bc = sum(mae_bc) / len(mae_bc)
    avg_mae_ec = sum(mae_ec) / len(mae_ec)
    avg_mae_pc = sum(mae_pc) / len(mae_pc)

    # Concatenate flattened matrices
    pred_1d = np.concatenate(pred_1d_list)
    gt_1d = np.concatenate(gt_1d_list)

    # Compute metrics
    mae = mean_absolute_error(pred_1d, gt_1d)
    pcc = pearsonr(pred_1d, gt_1d)[0]
    js_dis = jensenshannon(pred_1d, gt_1d)

    print("MAE: ", mae)
    print("PCC: ", pcc)
    print("Jensen-Shannon Distance: ", js_dis)
    print("Average MAE betweenness centrality:", avg_mae_bc)
    print("Average MAE eigenvector centrality:", avg_mae_ec)
    print("Average MAE PageRank centrality:", avg_mae_pc)
    return mae, pcc, js_dis, avg_mae_bc, avg_mae_ec, avg_mae_pc

Kaggle evaluation

In [ ]:
def calculate_centralities(adj_matrix):
    if adj_matrix.shape[0] != adj_matrix.shape[1]:
        raise ValueError(f"Adjacency matrix is not square: shape={adj_matrix.shape}")
    print(f"Processing adjacency matrix of shape: {adj_matrix.shape}")

    G = nx.from_numpy_array(adj_matrix)
    partition = community_louvain.best_partition(G)

    # Calculate the participation coefficient with the partition
    pc_dict = participation_coefficient(G, partition)

    # Calculate averages of centrality measures
    pr = nx.pagerank(G, alpha=0.9)
    ec = nx.eigenvector_centrality_numpy(G, max_iter=100)
    bc = nx.betweenness_centrality(G, normalized=True, endpoints=False)
    ns = np.array(list(nx.degree_centrality(G).values())) * (len(G.nodes()) - 1)
    acc = nx.average_clustering(G, weight=None)

    # Average participation coefficient
    pc_avg = np.mean(list(pc_dict.values()))

    return {
        'pr': np.mean(list(pr.values())),
        'ec': np.mean(list(ec.values())),
        'bc': np.mean(list(bc.values())),
        'ns': ns,
        'pc': pc_avg,
        'acc': acc
    }

def participation_coefficient(G, partition):
    # Initialize dictionary for participation coefficients
    pc_dict = {}

    # Calculate participation coefficient for each node
    for node in G.nodes():
        node_degree = G.degree(node)
        if node_degree == 0:
            pc_dict[node] = 0.0
        else:
            # Count within-module connections
            within_module_degree = sum(1 for neighbor in G[node] if partition[neighbor] == partition[node])
            # Calculate participation coefficient
            pc_dict[node] = 1 - (within_module_degree / node_degree) ** 2

    return pc_dict


def evaluate_all(true_hr_matrices, predicted_hr_matrices, output_path='11-randomCV.csv'):
    print(true_hr_matrices.shape)
    print(predicted_hr_matrices.shape)

    num_subjects = true_hr_matrices.shape[0]
    results = []

    for i in range(num_subjects):
        true_matrix = true_hr_matrices[i, :, :]
        pred_matrix = predicted_hr_matrices[i, :, :]

        print(f"Evaluating subject {i+1} with matrix shapes: true={true_matrix.shape}, pred={pred_matrix.shape}")

        if true_matrix.shape != pred_matrix.shape or true_matrix.shape[0] != true_matrix.shape[1]:
            print(f"Error: Matrix shape mismatch or not square for subject {i+1}: true={true_matrix.shape}, pred={pred_matrix.shape}")
            continue

        metrics = {
            'ID': i + 1,
            'MAE': mean_absolute_error(true_matrix.flatten(), pred_matrix.flatten()),
            'PCC': pearsonr(true_matrix.flatten(), pred_matrix.flatten())[0],
            'JSD': jensenshannon(true_matrix.flatten(), pred_matrix.flatten()),
        }

        true_metrics = calculate_centralities(true_matrix)
        pred_metrics = calculate_centralities(pred_matrix)

        for key in ['NS', 'PR', 'EC', 'BC', 'PC', 'ACC']:
            metrics[f'MAE in {key}'] = mean_absolute_error([true_metrics[key.lower()]], [pred_metrics[key.lower()]])

        results.append(metrics)

    df = pd.DataFrame(results)
    if not df.empty:
        # Check if the file exists to decide whether to write headers
        file_exists = os.path.isfile(output_path)

        df.to_csv(output_path, mode='a', header=not file_exists, index=False)
        print(f"Results appended to {output_path}.")
    else:
        print("No data to save.")


In [ ]:
class Args:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)

args_dict = {
    "padding": 0,
    "hidden_dim": 350,
    "hr_dim": 268,
    "lr_dim": 160,
    "lmbda": 16,
    "splits": 3,
    "lr": 1e-4,
    "epochs": 200
}

args = Args(args_dict)

if not os.path.exists('./Result'):
    os.mkdir('./Result/')

if not os.path.exists('./images'):
    os.mkdir('./images/')

splits = [
    (0, 1, 2),
    (0, 2, 1),
    (1, 2, 0)
]

ks = [0.9, 0.7, 0.6, 0.5]
fold_num = 0

for train_indices in splits:
    model = GSRNet(ks, args)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    fold_num += 1
    print("=====================================================")
    print(f"Fold {fold_num}/3 started.")

    if fold_num == 1:
      subjects_adj, test_adj, subjects_ground_truth, test_ground_truth, val_adj, val_ground_truth = (
          np.concatenate((subjects_adj_split_1, subjects_adj_split_2), axis=0),
          data_lr_split_3_reshaped,
          np.concatenate((subjects_labels_split_1, subjects_labels_split_2), axis=0),
          data_hr_split_3_reshaped,
          np.concatenate((test_adj_split_1, test_adj_split_2), axis=0),
          np.concatenate((test_labels_split_1, test_labels_split_2), axis=0),
      )
    elif fold_num == 2:
      subjects_adj, test_adj, subjects_ground_truth, test_ground_truth, val_adj, val_ground_truth = (
          np.concatenate((subjects_adj_split_1, subjects_adj_split_3), axis=0),
          data_lr_split_2_reshaped,
          np.concatenate((subjects_labels_split_1, subjects_labels_split_3), axis=0),
          data_hr_split_2_reshaped,
          np.concatenate((test_adj_split_1, test_adj_split_3), axis=0),
          np.concatenate((test_labels_split_1, test_labels_split_3), axis=0),
      )
    else:
      subjects_adj, test_adj, subjects_ground_truth, test_ground_truth, val_adj, val_ground_truth = (
          np.concatenate((subjects_adj_split_2, subjects_adj_split_3), axis=0),
          data_lr_split_1_reshaped,
          np.concatenate((subjects_labels_split_2, subjects_labels_split_3), axis=0),
          data_hr_split_1_reshaped,
          np.concatenate((test_adj_split_2, test_adj_split_3), axis=0),
          np.concatenate((test_labels_split_2, test_labels_split_3), axis=0),
      )

    epochs_loss, val_epochs_loss = train(model, optimizer, subjects_adj, subjects_ground_truth, args, val_adj, val_ground_truth)
    preds = test(model, test_adj, test_ground_truth, args)
    print("Evaluating...")
    mae, pcc, js_dis, avg_mae_bc, avg_mae_ec, avg_mae_pc = evaluation_metrics(len(test_ground_truth), preds, test_ground_truth)

    epoch_loss_df = pd.DataFrame({'epoch': list(range(1, len(epochs_loss) + 1)), 'loss': epochs_loss})
    epoch_loss_df.to_csv(f'11_fold_{fold_num}_loss_curve.csv', index=False)

    val_epoch_loss_df = pd.DataFrame({'epoch': list(range(1, len(val_epochs_loss) + 1)), 'loss': val_epochs_loss})
    val_epoch_loss_df.to_csv(f'11_fold_{fold_num}_eval_loss_curve.csv', index=False)

    # Plot training and validation loss for the current fold
    plt.figure(figsize=(10, 6))
    plt.plot(epoch_loss_df['epoch'], epoch_loss_df['loss'], label='Training Loss')
    plt.plot(val_epoch_loss_df['epoch'], val_epoch_loss_df['loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Loss Curves for Fold {fold_num}')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'images/loss_curve_fold_{fold_num}.png')
    plt.close()

    # Plot metrics for the current fold
    metric_names = ['MAE', 'PCC', 'JS Distance', 'Avg MAE BC', 'Avg MAE EC', 'Avg MAE PC']
    values = [mae, pcc, js_dis, avg_mae_bc, avg_mae_ec, avg_mae_pc]
    plot_data = pd.DataFrame({
    'Metric': metric_names,
    'Value': values
    })
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Metric', y='Value', data=plot_data, palette='viridis')
    plt.title(f'Metrics for Fold {fold_num}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"images/bar_plot_fold_{fold_num}.png")
    plt.show()
    print(f"Fold {fold_num} bar plot saved!")

    evaluate_all(test_ground_truth, preds, output_path=f'11_fold_{fold_num}_evaluation.csv')

    with open(f'predictions_fold_{fold_num}.csv', 'w') as submission_file:
        submission_file.write('ID,Predicted\n')
        model.eval().to(device)
        for i in range(test_data_np.shape[0]):
            vector = test_data_np[i]
            matrix = matrix_vec.anti_vectorize(vector, 160)
            matrix = torch.from_numpy(matrix).type(torch.FloatTensor).to(device)
            preds, a, b, c = model(matrix)
            preds = unpad(preds, args.padding)
            preds = preds.cpu().detach().numpy()
            vector = matrix_vec.vectorize(preds)
            for j, val in enumerate(vector, start=1):
                # Set val to 0 if it is NaN or negative
                val = 0 if np.isnan(val) or val < 0 else val
                submission_file.write(f'{i * len(vector) + j},{val}\n')
    print(f"Fold {fold_num} predictions written to file.")
    print(f"Fold {fold_num}/3 completed.")
    print("=====================================================")

print("All folds completed.")